### Issues: 
1. Method ```generate_candidates(Fk, k)``` produces duplicate candidates.  
2. Method ```subgraph_match(subgraph1, subgraph2)``` does not use Ullman's Algorithm as mentioned in the book.
3. Most methods are brute force. (at this point, so be it :P )

### To Do:
Method ```check_downward_closure(candidate)``` needs to be implemented for pruning candidates.  
Complete ```edge_based_join_growth(g, minsup)```


### Questions for Matteo:  
Why is checking for downward closure important?  
Is generating duplicate candidates okay?  
How exactly does edge-based join growth work? Is there always two candidates?  
Should we implement non-apriori algorithms e.g. FP-growth?  
Help finding datasets?  

## Setup Methods:

In [159]:
import snap
import glob
import itertools

def allgraphs():
    graph_list=[]
    G1 = snap.GenRndGnm(snap.PNGraph, 100, 500,False)
    G2 = snap.GenRndGnm(snap.PNGraph, 100, 500,False)
    G3 = snap.GenRndGnm(snap.PNGraph, 100, 500,False)
    G4 = snap.GenRndGnm(snap.PNGraph, 100, 500,False)
    G5 = snap.GenRndGnm(snap.PNGraph, 100, 500,False)
    
    graph_list=[G1, G2, G3, G4, G5]
    
    return graph_list

# return list of graphs in given directory
def get_graph_database(dir_path):
    
    
    return g

    
# return all graph edges as list
def list_graph(graph):
    graph_list = []
    
    if graph.GetEdges() == 0:
        for N in graph.Nodes():
            graph_list.append(N.GetId())
    else:
        for E in graph.Edges():
            curr_edge = (E.GetSrcNId(), E.GetDstNId())
            graph_list.append(curr_edge)
    
    return graph_list
         

# print graphs and their supports / print dict
def print_dict(D, opt):
    if opt == "graph":
        for graph in D.keys():
            print("Graph: {}, Support: {}".format(list_graph(graph), D[graph]))    
    else:
        for key in D.keys():
            print("Key : {} , Value : {}".format(key, D[key]))

# return true if given graphs are the same
def compare_graphs(G1, G2):
    
    if G1.GetNodes() != G2.GetNodes() or G1.GetEdges() != G2.GetEdges(): 
        return False
    
    for N in G1.Nodes():
        if not G2.IsNode(N.GetId()): return False
    
    for E in G1.Edges():
        if not G2.IsEdge(E.GetSrcNId(), E.GetDstNId()): return False
        
    return True   

## Node-based Join Growth:

In [163]:
# Node-based Join Growth

'''
NOTE: (get_all_node_supports) 
This method assumes that there are no label repitions in any of the graphs.
That is, none of the graphs have more than one node with ID x.
'''
# return dict with all nodes in g and their support
def get_all_node_supports(graph_database):
    NS = {}
    
    for graph in graph_database:
        for N in graph.Nodes():
            curr_node = N.GetId()
            if curr_node in NS:
                NS[curr_node] += 1
            else:
                NS[curr_node] = 1
    
    return NS


# return dict with frequent singleton graphs and their supports
def get_frequent_singleton_graphs(NS, minsup):
    F1 = {}
    
    for N in NS:
        if NS[N] >= minsup:
            subgraph = snap.TUNGraph.New() # create new graph
            subgraph.AddNode(N) # add frequent node
            F1[subgraph] = NS[N] # graph support = node support
    
    return F1


# return candidate by joining singletons
def join_singletons(subgraph1, subgraph2):
    c = snap.TUNGraph.New() # new candidate subgraph
    c = snap.ConvertGraph(type(subgraph1), subgraph1)
    
    c.AddNode(subgraph2.BegNI().GetId()) # add subgraph2 node to subgraph1
    c.AddEdge(subgraph1.BegNI().GetId(), subgraph2.BegNI().GetId()) # add edge between nodes
        
    return c

'''
NOTE: (subgraph_match)
The book recommends Ullman's Algorithm for this,
which is recursive. I am not a fan recursion so I made the following,
which might not be as efficient but does the job.
'''
# return node and edge if they are the only non-matching ones
def subgraph_match(Gq, G):
    nmE_f = False # non-matching node found
    nmN_f = False # non-matching edge found
    nmN = None # non_matching node
    nmE = None # non-matching edge

    res = False

    for E in Gq.Edges():
        if not G.IsEdge(E.GetSrcNId(), E.GetDstNId()):
            if nmE_f:
                nmE_f = False
                break
            else:
                nmE = (E.GetSrcNId(), E.GetDstNId())
                nmE_f = True
    
    if nmE_f:
        for N in Gq.Nodes():
            if not G.IsNode(N.GetId()):
                if nmN_f:
                    nmN_f = False
                    break
                else:
                    nmN = N.GetId()
                    if nmN in nmE:
                        nmN_f = True
                    else:
                        break
                        
    if nmN_f and nmE_f:
        res = True
    
    return res, nmE, nmN

# return true if all k-1 subgraphs of candidate are in Fk
def prune_candidate(c, Fk):
    all_nodes = []
    
    for N in c.Nodes():
        all_nodes.append(N.GetId())
    
    # generate all node subsets
    kMinus1_subsets = list(itertools.combinations(all_nodes, len(all_nodes)-1))
    
    for subset in kMinus1_subsets:
        subgraph = c.GetSubGraph(list(subset))
        
        for graph in Fk:
            if compare_graphs(graph, subgraph):
                return True
    return False


# return candidates by performing node-based joins    
def join_subgraphs(subgraph1, subgraph2, nmE, nmN):
    
    # create new candidate subgraphs
    c1 = snap.TUNGraph.New()
    c2 = snap.TUNGraph.New()
    
    # hold non-matching node in subgraph2
    for N in subgraph2.Nodes():
        if not subgraph1.IsNode(N.GetId()):
            nmN_s2 = N.GetId()
    
    c1 = snap.ConvertGraph(type(subgraph2), subgraph2) # copy subgraph1
    c1.AddNode(nmN) # add non-matching node from subgraph1
    c1.AddEdge(nmE[0], nmE[1]) # add non-matching edge
    
    c2 = snap.ConvertGraph(type(c1), c1) # copy candidate 1
    c2.AddEdge(nmN, nmN_s2) # add edge between non-matching nodes of subgraph1 and subgraph2
    
    return c1, c2

        
'''
NOTE: (generate_candidates)
This method is incomplete i.e. no optimization/pruning of candidates
'''
# return candidates list by joining graphs in Fk
def generate_candidates(Fk, k):
    candidates = []
    
    for i in range(0, len(Fk)):
        sG1 = Fk[i]
        
        for j in range(i+1, len(Fk)):
            sG2 = Fk[j]
            
            # if graphs are from F1, simply join them
            if k == 2:
                c = join_singletons(sG1, sG2)
                candidates.append(c)
            
            # if graphs are from Fk where k > 1,
            else:
                # join if they each have only one uncommon node
                match, nmE, nmN = subgraph_match(sG1, sG2)
                if match:
                    c1, c2 = join_subgraphs(sG1, sG2, nmE, nmN)
                    for _c in c1, c2:
                        if prune_candidate(_c, Fk):
                            candidates.append(_c)
    
    return candidates
        
    
# generate frequent k+1 sized graphs dict by counting C in g
def generate_Fkplus1(C, g, minsup):
    Fkplus1 = {}
    candidate_is_subgraph = True
    support = 0
    
    for candidate in C:
        for graph in g:

            # check if graph contains all candidate's nodes
            for N in candidate.Nodes():
                if not graph.IsNode(N.GetId()):
                    candidate_is_subgraph = False
                    break

            # check if graph contains all candidate's edges    
            if candidate_is_subgraph:        
                for E in candidate.Edges():
                    if not graph.IsEdge(E.GetSrcNId(), E.GetDstNId()):
                        candidate_is_subgraph = False
                        break

            # increment support            
            if candidate_is_subgraph:
                support += 1

            # reset for next graph
            candidate_is_subgraph = True

        if support >= minsup:
            Fkplus1[candidate] = support

        support = 0 # reset support for next candidate
    
    return Fkplus1


'''
NOTE: (update_FsG)
This method is not optimized because it is not necessary.
But it is used because if there cycles in the graphs,
duplicate entries are appended to FsG
'''
# return FsG after appending subgraphs not in FsG from Fk
def update_FsG(Fk, FsG):
    exists = False
    
    # compare each graph in Fk with each graph in FsG
    for g in Fk.keys(): 
        
        for G in FsG.keys():
            
            if compare_graphs(g, G):
                exists = True
                
        if not exists:
            FsG[g] = Fk[g]
            
        exists = False
        
    return FsG


# return dict with frequent subgraphs and support
def node_based_join_growth(g, minsup):
    
    # NS = { [(NodeId) : Support] }
    NS = get_all_node_supports(g)
    
    # F1 = { All frequent singleton graphs }
    Fk = get_frequent_singleton_graphs(NS, minsup) # frequent k subgraphs
    k = 1
    
    FsG = {} # all frequent subgraphs
    C = [] # candidates from Fk
    
    # Apriori Algorithm:
    while(True):
        C = generate_candidates(list(Fk.keys()), k+1)
        Fk = generate_Fkplus1(C, g, minsup)
        
        # end if no more frequent subgraphs
        if not Fk: 
            break
            
        FsG = update_FsG(Fk, FsG) # append all frequent subgraphs
        
        k = k + 1
    
    return FsG

## Edge-based Join Growth:

In [199]:
# Edge-based Join Growth

'''
NOTE: (get_all_edge_supports)
This method takes duplication into account.
That is, the edges NodeX-NodeY and NodeY-NodeX are considered the same.
''' 
# return dict will all edges in graph database and their supports
def get_all_edge_supports(graph_database):
        ES = {}    
        
        for graph in graph_database:
            for E in graph.Edges():
                curr_edge = (E.GetSrcNId(), E.GetDstNId())
                curr_edge_flip = (E.GetDstNId(), E.GetSrcNId())
                if curr_edge in ES:
                    ES[curr_edge] += 1
                elif curr_edge_flip in ES:
                    ES[curr_edge_flip] += 1
                else:    
                    ES[curr_edge] = 1
        
        return ES


# return all edges that have at least minsup support   
def get_frequent_singleton_graphs_EB(ES, minsup):
    F1 = {}
    
    for E in ES:
        if ES[E] >= minsup:
            subgraph = snap.TUNGraph.New()
            subgraph.AddNode(E[0])
            subgraph.AddNode(E[1])
            subgraph.AddEdge(E[0], E[1])
            F1[subgraph] = ES[E]

    return F1


'''
NOTE: (subgraph_match_EB)
The book recommends Ullman's Algorithm for this,
which is recursive. I am not a fan recursion so I made the following,
which might not be as efficient but does the job.
'''
# return node and edge if they are the only non-matching ones
def subgraph_match_EB(sG1, sG2): 
    nmE_f = False # non-matching node found
    nmE = None # non-matching edge
    
    for E in sG1.Edges():
        if not sG2.IsEdge(E.GetSrcNId(), E.GetDstNId()):
            if nmE_f:
                nmE_f = False
                break
            else:
                nmE = (E.GetSrcNId(), E.GetDstNId())
                nmE_f = True
    
    if nmE_f:
        if sG2.IsNode(nmE[0]) and sG2.IsNode(nmE[1]):
            return True, nmE, None
        elif sG2.IsNode(nmE[0]):
            return True, nmE, nmE[1]
        elif sG2.IsNode(nmE[1]):
            return True, nmE, nmE[0]
    
    return False, None, None
    

# return candidates by performing node-based joins        
def join_subgraphs_EB(subgraph1, subgraph2, nmE, nmN):
    
    c = snap.ConvertGraph(type(subgraph2), subgraph2)

    if nmN is not None:
        c.AddNode(nmN)

    c.AddEdge(nmE[0], nmE[1])
    
    return c


# return true if all k-1 subgraphs of candidate are in Fk
def prune_candidate_EB(c, Fk):
    all_nodes = []
    
    for N in c.Nodes():
        all_nodes.append(N.GetId())
    
    # generate all node subsets
    kMinus1_subsets = list(itertools.combinations(all_nodes, len(all_nodes)-1))
    
    for subset in kMinus1_subsets:
        subgraph = c.GetSubGraph(list(subset))
        
        for graph in Fk:
            if compare_graphs(graph, subgraph):
                return True
    return False
       
    
# return candidates list by joining graphs in Fk
def generate_candidates_EB(Fk, k):
    candidates = []
    
    for i in range(0, len(Fk)):
        sG1 = Fk[i]
        
        for j in range(i+1, len(Fk)):
            sG2 = Fk[j]
            match, nmE, nmN = subgraph_match_EB(sG1, sG2)
            if match:
                c = join_subgraphs_EB(sG1, sG2, nmE, nmN)
                if prune_candidate(c, Fk):
                    candidates.append(c)
                
    return candidates


# generate frequent k+1 sized graphs dict by counting C in g
def generate_Fkplus1_EB(C, g, minsup): 
    Fkplus1 = {}
    candidate_is_subgraph = True
    support = 0
    
    for candidate in C:
        for graph in g:

            # check if graph contains all candidate's nodes
            for N in candidate.Nodes():
                if not graph.IsNode(N.GetId()):
                    candidate_is_subgraph = False
                    break

            # check if graph contains all candidate's edges    
            if candidate_is_subgraph:        
                for E in candidate.Edges():
                    if not graph.IsEdge(E.GetSrcNId(), E.GetDstNId()):
                        candidate_is_subgraph = False
                        break

            # increment support            
            if candidate_is_subgraph:
                support += 1

            # reset for next graph
            candidate_is_subgraph = True

        if support >= minsup:
            Fkplus1[candidate] = support

        support = 0 # reset support for next candidate
    
    return Fkplus1


'''
NOTE: (update_FsG)
This method is not optimized because it is not necessary.
But it is used because if there cycles in the graphs,
duplicate entries are appended to FsG
'''
# return FsG after appending subgraphs not in FsG from Fk
def update_FsG_EB(Fk, FsG):
    exists = False

    for g in Fk.keys(): 
        for G in FsG.keys():
            
            if compare_graphs(g, G):
                exists = True
                
        if not exists:
            FsG[g] = Fk[g]
            
        exists = False
        
    return FsG


# return dict with frequent subgraphs and support
def edge_based_join_growth(g, minsup):
    
    # ES = { [ [(SrcNodeId), (DstNodeId) : Support] }
    ES = get_all_edge_supports(g)
    print_dict(ES, "")
    # F1 = { All frequent singleton graphs }
    Fk = get_frequent_singleton_graphs_EB(ES, minsup)
    k = 1
    
    FsG = Fk # all frequent subgraphs
    C = [] # candidates from Fk
    
    # Apriori Algotithm
    while(True):
        C = generate_candidates_EB(list(Fk.keys()), k+1)
        Fk = generate_Fkplus1_EB(C, g, minsup)
        
        # end if no more frequent subgraphs
        if not Fk:
            break
        FsG = update_FsG_EB(Fk, FsG) # append all frequent subgraphs
        
        k = k + 1
        
    return FsG

## Main:

In [213]:
# minsup = minimum support
minsup = 3

# g = [G1, G2, G3 ... Gn] 
g = get_graph_database("./datasets/test-graphs2/*.txt")
# g = allgraphs()

In [214]:
# FsG = Frequent subgraphs (with Node-based Join Growth)
FsG = node_based_join_growth(g, minsup)

# Print Results
print_dict(FsG, "graph")
print("Total: ", len(FsG))

Graph: [(0, 40)], Support: 3
Graph: [(0, 95)], Support: 3
Graph: [(2, 35)], Support: 3
Graph: [(2, 89)], Support: 3
Graph: [(3, 48)], Support: 3
Graph: [(5, 12)], Support: 3
Graph: [(6, 17)], Support: 3
Graph: [(6, 84)], Support: 3
Graph: [(8, 55)], Support: 3
Graph: [(8, 76)], Support: 3
Graph: [(11, 20)], Support: 3
Graph: [(11, 74)], Support: 3
Graph: [(13, 59)], Support: 3
Graph: [(13, 93)], Support: 3
Graph: [(14, 93)], Support: 3
Graph: [(15, 34)], Support: 4
Graph: [(15, 54)], Support: 3
Graph: [(17, 50)], Support: 3
Graph: [(18, 46)], Support: 3
Graph: [(19, 63)], Support: 3
Graph: [(21, 61)], Support: 3
Graph: [(22, 29)], Support: 3
Graph: [(22, 97)], Support: 3
Graph: [(23, 92)], Support: 3
Graph: [(23, 98)], Support: 3
Graph: [(25, 49)], Support: 3
Graph: [(27, 52)], Support: 3
Graph: [(29, 59)], Support: 3
Graph: [(35, 39)], Support: 3
Graph: [(36, 70)], Support: 4
Graph: [(40, 41)], Support: 3
Graph: [(42, 60)], Support: 3
Graph: [(46, 94)], Support: 3
Graph: [(53, 93)], S

In [201]:
# FsG_EB = Frequent subgraphs (with Edge-based Join Growth)
FsG_EB = edge_based_join_growth(g, minsup)

# Print Results
print_dict(FsG_EB, "graph")
print("Total: ", len(FsG_EB))

Key : (0, 7) , Value : 2
Key : (0, 8) , Value : 2
Key : (0, 15) , Value : 2
Key : (0, 20) , Value : 2
Key : (0, 27) , Value : 2
Key : (0, 30) , Value : 4
Key : (0, 45) , Value : 2
Key : (0, 62) , Value : 4
Key : (0, 68) , Value : 2
Key : (0, 88) , Value : 2
Key : (0, 93) , Value : 2
Key : (1, 10) , Value : 4
Key : (1, 50) , Value : 2
Key : (1, 53) , Value : 2
Key : (1, 60) , Value : 2
Key : (1, 65) , Value : 2
Key : (1, 66) , Value : 4
Key : (1, 75) , Value : 4
Key : (1, 84) , Value : 2
Key : (1, 95) , Value : 4
Key : (2, 18) , Value : 2
Key : (2, 22) , Value : 4
Key : (2, 25) , Value : 4
Key : (2, 29) , Value : 4
Key : (2, 44) , Value : 4
Key : (2, 68) , Value : 4
Key : (2, 85) , Value : 2
Key : (3, 16) , Value : 6
Key : (3, 22) , Value : 2
Key : (3, 41) , Value : 4
Key : (3, 43) , Value : 2
Key : (3, 62) , Value : 6
Key : (3, 73) , Value : 2
Key : (3, 74) , Value : 2
Key : (4, 16) , Value : 4
Key : (4, 23) , Value : 2
Key : (4, 24) , Value : 2
Key : (4, 38) , Value : 2
Key : (4, 44) 

Key : (73, 93) , Value : 2
Key : (73, 99) , Value : 2
Key : (74, 82) , Value : 2
Key : (74, 90) , Value : 2
Key : (74, 93) , Value : 2
Key : (75, 81) , Value : 2
Key : (77, 93) , Value : 2
Key : (77, 97) , Value : 2
Key : (78, 80) , Value : 2
Key : (78, 91) , Value : 2
Key : (80, 82) , Value : 2
Key : (80, 89) , Value : 2
Key : (81, 86) , Value : 2
Key : (81, 90) , Value : 2
Key : (83, 86) , Value : 2
Key : (83, 99) , Value : 2
Key : (85, 87) , Value : 2
Key : (85, 96) , Value : 2
Key : (88, 89) , Value : 2
Key : (88, 97) , Value : 2
Key : (88, 99) , Value : 2
Key : (89, 93) , Value : 2
Key : (89, 98) , Value : 2
Key : (90, 97) , Value : 2
Key : (91, 93) , Value : 2


### Trials and Testing:

In [ ]:
subgraph1 = snap.TUNGraph.New()
subgraph1.AddNode(1)
subgraph1.AddNode(6)
subgraph1.AddNode(7)
subgraph1.AddEdge(1, 6)
subgraph1.AddEdge(1, 7)

subgraph2 = snap.TUNGraph.New()
subgraph2.AddNode(1)
subgraph2.AddNode(12)
subgraph2.AddNode(5)
subgraph2.AddEdge(1, 12)
subgraph2.AddEdge(1, 5)

res = subgraph_match(subgraph1, subgraph2)

print(res)

#-------EB Debug:

# FsG_EB
ES = get_all_edge_supports(g)
Fk = get_frequent_singleton_graphs_EB(ES, 4)

print_dict(Fk, "graph")

In [ ]:
import glob
graph_paths = glob.glob('datasets/test-graphs/*.txt')
print(graph_paths)